# 의대 프로젝트 외래 경과() 파싱 (하이라이트)

In [2]:
import pandas as pd
import json
import numpy as np
import io
import math

In [3]:
import re

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    return [atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text)]

In [4]:
df_afters = pd.DataFrame(pd.read_csv('220405goutdata/entire_cases.csv'))[['케이스번호', 'dtype', '서식내용']]
condition = (df_afters.dtype == 'afters')
df_afters = df_afters[condition]

df_afters = df_afters.groupby('케이스번호', as_index= False)['서식내용'].apply(lambda x: '\n'.join(x))
test1 = df_afters.iloc[0]['서식내용']
test2 = df_afters.iloc[1]['서식내용']
test3 = df_afters.iloc[2]['서식내용']
test4 = df_afters.iloc[3]['서식내용']

df_afters.head()

/var/folders/st/n7vhj1ln7bxdtvslhvp85fw40000gn/T/ipykernel_43729/2029699529.py:1: DtypeWarning: Columns (5,6,7,10,11,12,13,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_afters = pd.DataFrame(pd.read_csv('220405goutdata/entire_cases.csv'))[['케이스번호', 'dtype', '서식내용']]


,케이스번호,서식내용
0,Case 1,"2021.2.15 초진 -> colc, feburic 처방. statin + nor..."
1,Case 10,rt ankle 많이 가라앉았다\n\nvimovo 좀 드심\n\n\n\n\nurin...
2,Case 2,attack (-)\n\n\n\n\nurinon 유지\n\n6달후\n\n\n\n\n...
3,Case 21,2019.4.2 초진\n\nfeburic 다시 복용하면서 검사하심\n\n\n\n\n...
4,Case 22,attack (-)\n\n6월에 요로결석으로 시술해서 뺐다\n\n\n\n\nfebu...


#### 텍스트 전처리

In [6]:
def split_by_line(string):
    newString = string.split('\n')
    remove = ['', '－', '＋']
    newString = list(filter(lambda val: val.strip() not in remove and len(val) > 1, newString))
    # newString = (',').join(newString)
    return newString

Ltest1 = split_by_line(test1)
Ltest1, len(Ltest1)

(['2021.2.15 초진 -> colc, feburic 처방. statin + norvasc는 있으심',
  'Gout',
  'HTN with hyperlipidemia on med',
  '(2/15)',
  'ESR 8',
  'Uric acid 8.0',
  'AST(GOT) 28',
  'ALT(GPT) 44',
  'Creatinine 1.00',
  'hs-CRP 0.07',
  '2021-02-23',
  '검사명 : Foot Both AP',
  '[Conclusion]',
  'No significant bony abnormality',
  '약 잘 드심',
  'attack (-)',
  '약 다 먹었다',
  '2달후',
  'gout',
  'HTN with hyperlipidemia on med',
  'feburic, colc 유지',
  'attack (-)',
  '잘 지내심',
  '고혈압약을 최근 잠깐 바궈먹었다',
  '약 일단 유지',
  '1달후',
  'gout',
  'HTN with hyperlipidemia on med',
  'Uric acid 3.9',
  'feburic, colc 유지',
  'attack (-)',
  '체중 비슷하다',
  'feburic 잠시 hold해 보겠습니다.',
  'colc, rosuampin 유지',
  '6주후',
  'gout',
  'HTN with hyperlipidemia on med',
  'AST(GOT) 59',
  'ALT(GPT) 109',
  'Uric acid 4.2',
  'TG 278',
  'HDL Chol. 35',
  'LDL Chol.(계산식) 75',
  'feburic 잠시 hold. colc, rosuampin 유지',
  'attack (-)',
  'colc만 유지',
  'LFT f/u후 다시 feburic만 시작 예정',
  '2주후',
  'gout',
  'HTN with hyperlipidemia on med',
  'AS

단순하게 생각해서 모든 문장을 나눈 이후 그 문장이 요약 소견서에 들어가있는지 안들어가 있는지를 라벨링한 데이터 프레임 생성. 이후 모델을 비교하며 (감정분석처럼) 문장의 중요도 파악. 카테고리도 비슷하게 적용시켜 볼 수 있을 것 같음.  

In [77]:
# import itertools 

# sents = []
# for i in range(len(df_firsts)):
#     text = df_firsts.iloc[i]['서식내용']
#     sent = split_by_line(text)
#     sents.append(sent)

# sentlist = list(itertools.chain.from_iterable(sents))
# sents_df = pd.DataFrame({'문장': sentlist, '유무': np.nan, '카테고리': np.nan})

# sents_df.to_csv('firsts_sentences.csv')

In [62]:
from konlpy.tag import Okt
okt = Okt()

textList = []

for i in range(len(Ltest1) - 1):
    text = Ltest1[i]
    parsed = okt.morphs(text, stem=True)
    textList.append(parsed)

### Text Rank 구현
https://github.com/lovit/textrank/ 참고

아래는 모두 textrank 구현본 
-> TextRank를 사용한 핵심문장 추출 

In [56]:
from collections import Counter
from collections import defaultdict
from scipy.sparse import csr_matrix

In [57]:
#tokenize: okt or mecab

def scan_vocabulary(sents, tokenizer):
    counter = Counter(w for sent in sents for w in tokenizer(sent))
    counter = {w: c for w, c in counter.items()}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

In [58]:
def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))


def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurence=2):
    counter = defaultdict(int)
    for s, token_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in token_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 9:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k, v in counter.items() if v >= min_cooccurence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)

In [46]:
def word_graph(sents, tokenize, window, min_cooccurrence):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize)
    tokens = [tokenize(sent) for sent in sents]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence)
    return g, idx_to_vocab

tokenize = okt.morphs
window = 2
min_cooccurence = 2

# word_graph(Ltest1, tokenize, window, min_cooccurence)

In [64]:
from sklearn.preprocessing import normalize

def pagerank(x, df, max_iter):
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

def textrank_keyword(sents, tokenize, window, min_cooccurrence, df, max_iter, topk):
    g, idx_to_vocab = word_graph(sents, tokenize, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keywords = [(idx_to_vocab[idx], R[idx]) for idx in reversed(idxs)]
    return keywords

df = 0.70
topk = 30
max_iter = 30

# textrank_keyword(Ltest1, tokenize, window, min_cooccurence, df, max_iter, topk)

In [ ]:
from scipy.sparse import csr_matrix

def sent_graph(sents, tokenize, similarity, min_count=2, min_sim=0.3):
    _, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)

    tokens = [[w for w in tokenize(sent) if w in vocab_to_idx] for sent in sents]
    rows, cols, data = [], [], []
    n_sents = len(tokens)
    for i, tokens_i in enumerate(tokens):
        for j, tokens_j in enumerate(tokens):
            if i >= j:
                continue
            sim = similarity(tokens_i, tokens_j)
            if sim < min_sim:
                continue
            rows.append(i)
            cols.append(j)
            data.append(sim)
    return csr_matrix((data, (rows, cols)), shape=(n_sents, n_sents))

def textrank_sent_sim(s1, s2):
    n1 = len(s1)
    n2 = len(s2)
    if (n1 <= 1) or (n2 <= 1):
        return 0
    common = len(set(s1).intersection(set(s2)))
    base = math.log(n1) + math.log(n2)
    return common / base

def cosine_sent_sim(s1, s2):
    if (not s1) or (not s2):
        return 0

    s1 = Counter(s1)
    s2 = Counter(s2)
    norm1 = math.sqrt(sum(v ** 2 for v in s1.values()))
    norm2 = math.sqrt(sum(v ** 2 for v in s2.values()))
    prod = 0
    for k, v in s1.items():
        prod += v * s2.get(k, 0)

In [ ]:
class KeywordSummarizer:
    def __init__(self, sents=None, tokenize=None, min_count=2,
        window=-1, min_cooccurrence=2, vocab_to_idx=None,
        df=0.85, max_iter=30, verbose=False):

        self.tokenize = tokenize
        self.min_count = min_count
        self.window = window
        self.min_cooccurrence = min_cooccurrence
        self.vocab_to_idx = vocab_to_idx
        self.df = df
        self.max_iter = max_iter
        self.verbose = verbose

        if sents is not None:
            self.train_textrank(sents)

    def train_textrank(self, sents, bias=None):
        g, self.idx_to_vocab = word_graph(sents,
            self.tokenize, self.min_count,self.window,
            self.min_cooccurrence, self.vocab_to_idx, self.verbose)
        self.R = pagerank(g, self.df, self.max_iter, bias).reshape(-1)
        if self.verbose:
            print('trained TextRank. n words = {}'.format(self.R.shape[0]))

    def keywords(self, topk=30):
        if not hasattr(self, 'R'):
            raise RuntimeError('Train textrank first or use summarize function')
        idxs = self.R.argsort()[-topk:]
        keywords = [(self.idx_to_vocab[idx], self.R[idx]) for idx in reversed(idxs)]
        return keywords

    def summarize(self, sents, topk=30):
        self.train_textrank(sents)
        return self.keywords(topk)


class KeysentenceSummarizer:
    def __init__(self, sents=None, tokenize=None, min_count=2,
        min_sim=0.3, similarity=None, vocab_to_idx=None,
        df=0.85, max_iter=30, verbose=False):

        self.tokenize = tokenize
        self.min_count = min_count
        self.min_sim = min_sim
        self.similarity = similarity
        self.vocab_to_idx = vocab_to_idx
        self.df = df
        self.max_iter = max_iter
        self.verbose = verbose

        if sents is not None:
            self.train_textrank(sents)

    def train_textrank(self, sents, bias=None):
        g = sent_graph(sents, self.tokenize, self.min_count,
            self.min_sim, self.similarity, self.vocab_to_idx, self.verbose)
        self.R = pagerank(g, self.df, self.max_iter, bias).reshape(-1)
        if self.verbose:
            print('trained TextRank. n sentences = {}'.format(self.R.shape[0]))

    def summarize(self, sents, topk=30, bias=None):
        n_sents = len(sents)
        if isinstance(bias, np.ndarray):
            if bias.shape != (n_sents,):
                raise ValueError('The shape of bias must be (n_sents,) but {}'.format(bias.shape))
        elif bias is not None:
            raise ValueError('The type of bias must be None or numpy.ndarray but the type is {}'.format(type(bias)))

        self.train_textrank(sents, bias)
        idxs = self.R.argsort()[-topk:]
        keysents = [(idx, self.R[idx], sents[idx]) for idx in reversed(idxs)]
        return keysents